In [1]:
import pathlib

import pandas as pd
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import coalesce, desc, first, last, lead, lower, lpad, row_number, max, min, upper, year
from pyspark.sql.types import StructField, StructType, ByteType, DateType, FloatType, IntegerType, ShortType, StringType
import tqdm

from utils.regions import Regions

In [2]:
pd.set_option("display.max_columns", None)

# Spark Configuration

In [3]:
conf = SparkConf()
conf.setMaster("local[*]").setAppName("Dataset Maker")

In [4]:
sc = SparkContext(conf=conf)

23/11/24 12:12:26 WARN Utils: Your hostname, asus-notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.186 instead (on interface wlp3s0)
23/11/24 12:12:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/24 12:12:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc.uiWebUrl

'http://192.168.1.186:4040'

In [6]:
session = SparkSession(sc)

# Small&Meduim Business Data

In [7]:
schema = StructType([
    StructField("data_date", DateType(), False),
    StructField("ind_tin", StringType(), True),
    StructField("org_tin", StringType(), True),
    StructField("region_name", StringType(), True),
    StructField("activity_code_main", StringType(), False), 
])
path = pathlib.Path("rsmp/reestr_test")
files = [str(fn) for fn in path.glob("data-*.csv")]
raw_data = session.read.options(
    header=True, dateFormat="dd.MM.yyyy", escape='"'
).schema(schema).csv(files)
raw_data.printSchema()

root
 |-- data_date: date (nullable = true)
 |-- ind_tin: string (nullable = true)
 |-- org_tin: string (nullable = true)
 |-- region_name: string (nullable = true)
 |-- activity_code_main: string (nullable = true)



In [8]:
raw_data.count()

41323453

In [9]:
raw_data.limit(10).toPandas()

,data_date,ind_tin,org_tin,region_name,activity_code_main
0,2020-12-10,None,1435279920,САХА /ЯКУТИЯ/,73.11
1,2020-12-10,143528320901,None,САХА /ЯКУТИЯ/,47.21
2,2020-12-10,None,1435283109,САХА /ЯКУТИЯ/,41.20
3,2020-12-10,143528826543,None,САХА /ЯКУТИЯ/,47.21
4,2020-12-10,143527925132,None,ПРИМОРСКИЙ,56.10.1
5,2020-12-10,143528630597,None,Саха /Якутия/,56.10
6,2020-12-10,143528107482,None,САХА /ЯКУТИЯ/,43.99.6
7,2020-12-10,None,1435280820,САХА /ЯКУТИЯ/,47.82
8,2020-12-10,None,1435287738,САХА /ЯКУТИЯ/,69.20
9,2020-12-10,None,1435285226,САХА /ЯКУТИЯ/,71.1


In [10]:
excluded_regions = [
    "Крым",
    "Севастополь",
    "Донецкая",
    "Луганская",
    "Запорожская",
    "Херсонская"
]
excluded_regions_condition = (
    "not ("
    + " or ".join(f"region_name ilike '%{region.upper()}%'" for region in excluded_regions)
    + ")"
)

reestr_data = (
    raw_data
    .filter(excluded_regions_condition)
    .filter("org_tin is not null")
    .withColumns({
        "org_tin": lpad("org_tin", 10, "0"),
        "year": year("data_date"),
    })
    .withColumnRenamed("org_tin", "tin")
    .select("region_name", "tin", "year", "activity_code_main")
    .dropDuplicates()
    .groupBy(["region_name", "year", "activity_code_main"])
    .count()
    .cache()
)
reestr_data.count()

599132

In [11]:
reestr_data.limit(10).toPandas()

,region_name,year,activity_code_main,count
0,АРХАНГЕЛЬСКАЯ,2020,47.11,200
1,ВЛАДИМИРСКАЯ,2020,46.34,16
2,ВОЛГОГРАДСКАЯ,2020,52.21.24,308
3,ИВАНОВСКАЯ,2020,14.12,272
4,КАЛИНИНГРАДСКАЯ,2020,46.17.1,31
5,КАЛИНИНГРАДСКАЯ,2020,16.10,59
6,КРАСНОДАРСКИЙ,2020,71.1,346
7,МОСКВА,2020,68.20,8998
8,МОСКВА,2020,47.52.73,155
9,МОСКВА,2020,78.30,316


In [12]:
reestr_data = reestr_data.toPandas()

In [40]:
rosstat_data = pd.read_csv("assets/rosstat-org-stats.csv", skiprows=2)
rosstat_data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2017,2018,2019,2020,2021,2022
0,NaN,NaN,NaN,на 1 октября,на 1 октября,на 1 октября,на 1 октября,на 1 октября,на 1 октября
1,ВСЕГО (по всем формам собственности),Белгородская область,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",NaN,NaN,1023,991,965,923
2,ВСЕГО (по всем формам собственности),Белгородская область,"Растениеводство и животноводство, охота и ...",NaN,NaN,941,907,883,843
3,ВСЕГО (по всем формам собственности),Белгородская область,Лесоводство и лесозаготовки,NaN,NaN,21,22,20,20
4,ВСЕГО (по всем формам собственности),Белгородская область,Лесозаготовки,NaN,NaN,3,5,5,6


In [41]:
rosstat_data = rosstat_data.iloc[1:, 1:]
rosstat_data.rename(
    columns={"Unnamed: 1": "region_name", "Unnamed: 2": "activity"},
    inplace=True
)
rosstat_data.drop(columns=["2017", "2018"], inplace=True)
rosstat_data["region_name"] = rosstat_data["region_name"].str.strip()
rosstat_data["activity"] = rosstat_data["activity"].str.strip()
rosstat_data

,region_name,activity,2019,2020,2021,2022
1,Белгородская область,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО...",1023,991,965,923
2,Белгородская область,"Растениеводство и животноводство, охота и пред...",941,907,883,843
3,Белгородская область,Лесоводство и лесозаготовки,21,22,20,20
4,Белгородская область,Лесозаготовки,3,5,5,6
5,Белгородская область,Рыболовство и рыбоводство,61,62,62,60
...,...,...,...,...,...,...
3311,Чукотский автономный округ,"ДЕЯТЕЛЬНОСТЬ В ОБЛАСТИ КУЛЬТУРЫ, СПОРТА, ОРГАН...",36,36,37,39
3312,Чукотский автономный округ,ПРЕДОСТАВЛЕНИЕ ПРОЧИХ ВИДОВ УСЛУГ,112,112,113,111
3313,Чукотский автономный округ,ДЕЯТЕЛЬНОСТЬ ДОМАШНИХ ХОЗЯЙСТВ КАК РАБОТОДАТЕЛ...,0,0,0,0
3314,Чукотский автономный округ,ДЕЯТЕЛЬНОСТЬ ЭКСТЕРРИТОРИАЛЬНЫХ ОРГАНИЗАЦИЙ И ...,0,0,0,0


In [15]:
classifier = pd.read_csv("assets/activity_codes_classifier.csv")
classifier.head(2)

,group,code,name
0,A,,"СЕЛЬСКОЕ, ЛЕСНОЕ ХОЗЯЙСТВО, ОХОТА, РЫБОЛОВСТВО..."
1,A,01,"Растениеводство и животноводство, охота и пред..."


In [42]:
rosstat_data = (
    rosstat_data
    .merge(classifier, how="left", left_on="activity", right_on="name")
    .query("code.str.strip() == ''")
    .drop_duplicates(subset=["region_name", "activity"])
    .drop(columns=["activity", "code", "name"])
)
rosstat_data.head(2)

,region_name,2019,2020,2021,2022,group
0,Белгородская область,1023,991,965,923,A
8,Белгородская область,56,55,47,45,B


In [43]:
rosstat_data[["2019", "2020", "2021", "2022"]] = rosstat_data[["2019", "2020", "2021", "2022"]].astype(float)

In [44]:
rosstat_stats = rosstat_data.groupby(["region_name", "group"])[["2019", "2020", "2021", "2022"]].sum().reset_index()
rosstat_stats.head(3)

,region_name,group,2019,2020,2021,2022
0,Алтайский край,A,1894.0,1778.0,1700.0,1664.0
1,Алтайский край,B,110.0,114.0,110.0,118.0
2,Алтайский край,C,4470.0,4341.0,4101.0,4001.0


In [20]:
regions = Regions()

In [45]:
rosstat_stats["region"] = rosstat_stats["region_name"].apply(lambda x: regions.get(x).name)

manual_fixes = {
    "Город Москва столица Российской Федерации город федерального значения": "Москва",
    "Ненецкий автономный округ (Архангельская область)": "Ненецкий автономный округ",
    "Архангельская область (кроме Ненецкого автономного округа)": "Архангельская область",
    "Ханты-Мансийский автономный округ - Югра (Тюменская область)": "Ханты-Мансийский автономный округ - Югра",
    "Ямало-Ненецкий автономный округ (Тюменская область)": "Ямало-Ненецкий автономный округ",
    "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)": "Тюменская область",
}
for search, replace in manual_fixes.items():
    rosstat_stats.loc[rosstat_stats["region_name"] == search, "region"] = replace
rosstat_stats = rosstat_stats.loc[~rosstat_stats["region_name"].isin((" Архангельская область", "Тюменская область")), ["region", "group", "2019", "2020", "2021", "2022"]]
rosstat_stats = rosstat_stats.melt(id_vars=["region", "group"], var_name="year", value_name="count")
rosstat_stats.head()

,region,group,year,count
0,Алтайский край,A,2019,1894.0
1,Алтайский край,B,2019,110.0
2,Алтайский край,C,2019,4470.0
3,Алтайский край,D,2019,426.0
4,Алтайский край,E,2019,399.0


In [46]:
reestr_stats = reestr_data.merge(
    classifier,
    how="left",
    left_on="activity_code_main",
    right_on="code",
)[["region_name", "year", "group", "count"]]
reestr_stats = reestr_stats.groupby(["region_name", "year", "group"])[["count"]].sum().reset_index()
reestr_stats["region"] = reestr_stats["region_name"].apply(lambda x: regions.get(x).name)
reestr_stats = reestr_stats.groupby(["region", "year", "group"])[["count"]].sum().reset_index()
reestr_stats.head()

,region,year,group,count
0,Алтайский край,2016,A,1502
1,Алтайский край,2016,B,91
2,Алтайский край,2016,C,3836
3,Алтайский край,2016,D,256
4,Алтайский край,2016,E,226


In [47]:
rosstat_stats["year"] = rosstat_stats["year"].astype(int)

In [48]:
stats = reestr_stats.merge(rosstat_stats, how="left", on=["region", "year", "group"], suffixes=("_reestr", "_rosstat"))
stats.dropna(subset=["count_rosstat"], inplace=True)
stats["norm_diff"] = (stats["count_reestr"] - stats["count_rosstat"]) / (stats["count_reestr"] +  stats["count_rosstat"])
stats.head()

,region,year,group,count_reestr,count_rosstat,norm_diff
60,Алтайский край,2019,A,1424,1894.0,-0.141652
61,Алтайский край,2019,B,84,110.0,-0.134021
62,Алтайский край,2019,C,3829,4470.0,-0.077238
63,Алтайский край,2019,D,210,426.0,-0.339623
64,Алтайский край,2019,E,213,399.0,-0.303922


In [49]:
stats[["count_reestr", "count_rosstat"]].corr(method="spearman")

,count_reestr,count_rosstat
count_reestr,1.000000,0.816609
count_rosstat,0.816609,1.000000


In [50]:
stats["norm_diff"].describe()

count    6546.000000
mean       -0.325885
std         0.279451
min        -0.996992
25%        -0.440365
50%        -0.214250
75%        -0.117973
max         1.000000
Name: norm_diff, dtype: float64

In [51]:
stats[stats["norm_diff"] == 1]

,region,year,group,count_reestr,count_rosstat,norm_diff
4357,Московская область,2019,U,1,0.0,1.0
4378,Московская область,2020,U,1,0.0,1.0
8489,Рязанская область,2022,T,1,0.0,1.0


In [52]:
stats.drop(columns="norm_diff").melt(id_vars=["region", "year", "group"], var_name="source", value_name="count").to_csv("stats.csv")

In [82]:
(
    stats
    .query("group not in ('O', 'P', 'R', 'U')")
    .groupby(["year", "region"])[["count_reestr", "count_rosstat"]]
    .corr(method="spearman")
    .unstack()
    .reset_index()
    .iloc[:, [0, 1, 3]]
    .describe()
)

,year,count_reestr
,,count_rosstat
count,332.000000,332.000000
mean,2020.500000,0.898393
std,1.119722,0.076414
min,2019.000000,0.568487
25%,2019.750000,0.884559
50%,2020.500000,0.917157
75%,2021.250000,0.948529
max,2022.000000,0.995098


In [78]:
(
    stats
    .query("group not in ('O', 'P', 'R', 'U')")
    .groupby(["year", "group"])[["count_reestr", "count_rosstat"]]
    .corr(method="spearman")
    .unstack()
    .reset_index()
    .iloc[:, [0, 1, 3]]
)

,year,group,count_reestr
,,,count_rosstat
0,2019,A,0.920075
1,2019,B,0.981489
2,2019,C,0.994052
3,2019,D,0.977671
4,2019,E,0.972175
...,...,...,...
63,2022,M,0.997013
64,2022,N,0.993475
65,2022,Q,0.977295


In [77]:
(
    stats
    .groupby(["group"])[["count_reestr", "count_rosstat"]]
    .corr(method="spearman")
    .unstack()
    .reset_index()
    .iloc[:, [0, 1, 3]]
)

,group,count_reestr,count_rosstat
,,count_reestr,count_reestr
0,A,1.0,0.930054
1,B,1.0,0.984205
2,C,1.0,0.995432
3,D,1.0,0.979212
4,E,1.0,0.974761
5,F,1.0,0.991472
6,G,1.0,0.994161
7,H,1.0,0.995959
8,I,1.0,0.995329


In [63]:
stats.groupby(["region", "group"])[["count_reestr", "count_rosstat"]].corr(method="spearman").unstack().iloc[:, 1].describe()

count    1592.000000
mean        0.568342
std         0.584021
min        -1.000000
25%         0.400000
50%         0.800000
75%         1.000000
max         1.000000
Name: (count_reestr, count_rosstat), dtype: float64